In [ ]:
import os
os.environ["SDL_VIDEODRIVER"] = "dummy"
#os.putenv('SDL_VIDEODRIVER', 'fbcon')
import random
import pygame
import math
import tkinter as tk
from tkinter import messagebox
import sys
import matplotlib.pyplot as plt
import numpy as np
from skimage.color import rgb2gray
import torchvision.transforms as T
from collections import namedtuple,deque

import torch.nn as nn
from itertools import count
from PIL import Image
import torch
import torch.optim as optim
import torch.nn.functional as F

import time
import torch
import gc
#gc.collect()


class Cube(object):
    rows=20
    w=200

    def __init__(self,start,dirnx=0,dirny=1,color=(255,0,0)):
        #dirnx 0: dont move in x direction; 1: move right; -1: move left
        #dirny 0: dont move in y direction; 1: move down; -1 move up

        self.pos=start
        self.dirnx=dirnx
        self.dirny=dirny
        self.color=color


    def move(self,dirnx,dirny):
        self.dirnx=dirnx
        self.dirny=dirny
        self.pos=(self.pos[0]+self.dirnx,self.pos[1]+self.dirny)

    def draw(self,surface,eyes=False):
        dis=self.w//self.rows
        i=self.pos[0]
        j=self.pos[1]
        pygame.draw.rect(surface,self.color,(i*dis+1,j*dis+1,dis-2,dis-2))
        if eyes:
            center=dis//2
            radius=1
            circleMiddle=(i*dis+center-radius-1,j*dis+2)
            circleMiddle2 = (i * dis +center+ radius+1, j * dis + 2)
            pygame.draw.circle(surface,(0,0,0),circleMiddle,radius)
            pygame.draw.circle(surface, (0, 0, 0), circleMiddle2, radius)

# if __name__=='__main__':
#     width = 500
#     rows = 20
#     status=[]
#     actions=[]
#     rewards=[]
#     win = pygame.display.set_mode((500, 600))
#     dirnx, dirny = random.sample(((-1, 0), (1, 0), (0, 1), (0, -1)), 1)[0]
#     s = Snake((255, 0, 0), (random.randint(5, 15), random.randint(5, 15)), dirnx, dirny)
#     snack = Cube(randomSnack(rows, s), color=(0, 255, 0))
#     flag = True
#     clock = pygame.time.Clock()
#     k=5
#     steps=k-1
#     i=1
#     while flag and i:
#
#         pygame.time.delay(50)
#         clock.tick(10)
#         redrawWindow(win, rows, width, s, snack)
#
#         status.append(pygame.surfarray.array3d(win))
#         AgentInput=random.randint(0,3)
#         actions.append(AgentInput)
#
#         s.move('Agent',AgentInput)
#         print(steps)
#         if AgentInput==0:
#             print('up')
#         elif AgentInput==1:
#             print('down')
#         elif AgentInput==2:
#             print('left')
#         else:
#             print('right')
#         if s.body[0].pos == snack.pos:
#             rewards.append(1)
#
#             steps+=10
#             s.addCube()
#             snack = Cube(randomSnack(rows, s), color=(0, 255, 0))
#         else:
#
#
#             for x in range(len(s.body)):
#                 if s.body[x].pos in list(map(lambda z: z.pos, s.body[x + 1:])) or s.head.pos[0] < 0 or s.head.pos[0] > 19 or \
#                         s.head.pos[1] < 0 or s.head.pos[1] > 19 or not steps:
#                     rewards.append(-1)
#                     print('Score: ', len(s.body)-1)
#                     #message_box('You Lost', 'Play again')
#                     dirnx, dirny = random.sample(((-1, 0), (1, 0), (0, 1), (0, -1)), 1)[0]
#                     s.reset((random.randint(5, 15), random.randint(5, 15)), dirnx, dirny)
#                     status.append(None)
#                     steps=k+1
#                     i-=1
#                     break
#                 else:
#                     rewards.append(0)
#
#         steps -= 1
#         redrawWindow(win, rows, width, s, snack)
#     pygame.quit()
#
#     # k=1
#     # x=type(states[0])
#     # for img in states:
#     #     if type(img)==x:
#     #         plt.figure(k)
#     #         k+=1
#     #         plt.imshow(img.transpose(1,0,2))





# curr_state=[]
# action=[]
# reward=[]
# next_state=[]
# s=env.get_curr_frame()
# Transition=namedtuple('Transition',
#                         ('state', 'action', 'reward', 'next_state'))
# t=[]
# tp=type(s)
# for j in range(2):
#     s = env.get_curr_frame()
#     #print('j:' ,j)
#     for i in range(10):
#         #print('i: ',i)
#         env.render()
#         a=random.randint(0,3)
#         r,ns=env.step(a,'Agent')
#         memory.push(s,a,r,ns)
#         print(env.steps)
#         if ns is None:
#             print('reset')
#             env.reset()
#             break
#         s=ns
#pygame.quit()





# plt.figure()
# plt.imshow(curr_state_pytorch.squeeze(0).permute(1, 2, 0).numpy(),
#            interpolation='none')
# plt.title('Example extracted screen')
# plt.show()








In [ ]:
class Snake(object):
    body=[]
    turns={}
    def __init__(self,color,pos,dirnx,dirny):
        self.color=color
        self.dirnx, self.dirny = dirnx,dirny
        self.head = Cube(pos,dirnx,dirny)
        self.body.append(self.head)
    def move(self,mode='Player',AgentInput=0):

        #the mode can be Player or Agent, Agent is DQN
        #when the mode is Player, use keyboard to play
        #when the mode is Agent, the AgentInput will be 0,1,2,3 ->up, down, left, right

        if mode=='Player':
            for event in pygame.event.get():
                if event.type==pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                keys=pygame.key.get_pressed()
                #for key in keys:
                if keys[pygame.K_LEFT] and (len(self.body)==1 or (len(self.body)>1 and not (self.dirnx==1 and self.dirny==0))):
                    self.dirnx=-1
                    self.dirny=0
                    self.turns[self.head.pos[:]]=[self.dirnx,self.dirny]


                elif keys[pygame.K_RIGHT] and (len(self.body)==1 or (len(self.body)>1 and not (self.dirnx==-1 and self.dirny==0))):

                    self.dirnx=1
                    self.dirny=0
                    self.turns[self.head.pos[:]]=[self.dirnx,self.dirny]


                elif keys[pygame.K_UP] and (len(self.body) == 1 or (len(self.body)>1 and not (self.dirnx==0 and self.dirny==1))):
                    self.dirnx=0
                    self.dirny=-1
                    self.turns[self.head.pos[:]]=[self.dirnx,self.dirny]

                elif keys[pygame.K_DOWN] and (len(self.body) == 1 or (len(self.body)>1 and not (self.dirnx==0 and self.dirny==-1))):

                    self.dirnx=0
                    self.dirny=1
                    self.turns[self.head.pos[:]]=[self.dirnx,self.dirny]


        elif mode=='Agent':
            if AgentInput==0 and (len(self.body) == 1 or (len(self.body)>1 and not (self.dirnx==0 and self.dirny==1))) :
                self.dirnx=0
                self.dirny=-1
                self.turns[self.head.pos[:]]=[self.dirnx,self.dirny]
                
            elif AgentInput==1 and (len(self.body) == 1 or (len(self.body)>1 and not (self.dirnx==0 and self.dirny==-1))):
                self.dirnx = 0
                self.dirny = 1
                self.turns[self.head.pos[:]] = [self.dirnx, self.dirny]
                
            elif AgentInput==2 and (len(self.body)==1 or (len(self.body)>1 and not (self.dirnx==1 and self.dirny==0))):
                self.dirnx = -1
                self.dirny = 0
                self.turns[self.head.pos[:]] = [self.dirnx, self.dirny]
                
            elif AgentInput==3 and (len(self.body)==1 or (len(self.body)>1 and not (self.dirnx==-1 and self.dirny==0))):
                self.dirnx = 1
                self.dirny = 0
                self.turns[self.head.pos[:]] = [self.dirnx, self.dirny]

        else:
            print('Invalid mode, Player or Agent')

        for i,c in enumerate(self.body):
            #print('length of turns',len(self.turns))
            p=c.pos[:]
            if p in self.turns:
                turn=self.turns[p]
                c.move(turn[0],turn[1])
                if i==len(self.body)-1:
                    self.turns.pop(p)
            else:
                # if c.dirnx==-1 and c.pos[0]<=0: c.pos=(c.rows-1,c.pos[1])
                # elif c.dirnx==1 and c.pos[0]>=c.rows-1: c.pos=(0,c.pos[1])
                # elif c.dirny==1 and c.pos[1]>=c.rows-1:c.pos=(c.pos[0],0)
                # elif c.dirny==-1 and c.pos[1]<=0:c.pos=(c.pos[0],c.rows-1)
                # else: c.move(c.dirnx,c.dirny)
                c.move(c.dirnx, c.dirny)


    def reset(self,pos,dirnx,dirny):
        self.head=Cube(pos,dirnx,dirny)
        self.body=[]
        self.body.append(self.head)
        self.turns={}
        self.dirnx=dirnx
        self.dirny=dirny

    def addCube(self):
        tail=self.body[-1]
        dx,dy=tail.dirnx,tail.dirny
        if dx==1 and dy==0:self.body.append(Cube((tail.pos[0]-1,tail.pos[1])))
        elif dx==-1 and dy==0:self.body.append(Cube((tail.pos[0]+1,tail.pos[1])))
        elif dx == 0 and dy == 1:self.body.append(Cube((tail.pos[0], tail.pos[1]-1)))
        elif dx == 0 and dy == -1:self.body.append(Cube((tail.pos[0], tail.pos[1] + 1)))

        self.body[-1].dirnx=dx
        self.body[-1].dirny=dy

    def draw(self,surface):
        for i,c in enumerate(self.body):
            if i==0:
                c.draw(surface,True)
            else:
                c.draw(surface)



In [ ]:
class Environment(object):
    r=0.01

    def __init__(self,screen_width=500,screen_height=500,rows=20,steps=20,reward_steps=200):
        self.next_state=None
        self.score=0
        self.reward=0
        self.steps=steps
        self.stepcount=steps
        self.reward_steps=reward_steps
        self.screen_width=screen_width
        self.screen_height=screen_height
        self.win = pygame.display.set_mode((self.screen_width,self.screen_height))
        self.dirnx, self.dirny = random.sample(((-1, 0), (1, 0), (0, 1), (0, -1)), 1)[0]
        self.snake=Snake((255, 0, 0), (random.randint(5, 15), random.randint(5, 15)), self.dirnx, self.dirny)
        self.rows=rows
        self.snack = Cube(randomSnack(self.rows, self.snake), color=(0, 255, 0))
        self.stack=[]

    def reset(self):
        self.dirnx, self.dirny = random.sample(((-1, 0), (1, 0), (0, 1), (0, -1)), 1)[0]
        self.snake.reset((random.randint(5, 15), random.randint(5, 15)), self.dirnx, self.dirny)
        self.snack = Cube(randomSnack(self.rows, self.snake), color=(0, 255, 0))
        self.score=0
        self.steps=self.stepcount
        self.reward=0

    def render(self,display=False,fps=10000):
        clock = pygame.time.Clock()
        #pygame.time.delay(10)
        clock.tick(fps)

        self.win.fill((0,0,0))
        self.snake.draw(self.win)
        self.snack.draw(self.win)


        if display==True:
            pygame.display.set_caption('Score: '+str(self.score))
        pygame.display.update()




    def get_curr_frame(self):
        frame=pygame.surfarray.array3d(self.win)
        #frame=rgb2gray(frame)
        frame=frame.astype(np.uint8)
        frame=frame.transpose([1,0,2])
        return frame


    def step(self,action,mode='Player'):
        self.reward = 0
        self.snake.move(mode,action)
        if self.snake.body[0].pos == self.snack.pos:
            self.reward=2
            self.score+=self.reward

            self.steps+=self.reward_steps
            self.snake.addCube()
            self.snack = Cube(randomSnack(self.rows, self.snake), color=(0, 255, 0))
            self.render()
            self.next_state=self.get_curr_frame()
        else:
            self.render()
            self.next_state=self.get_curr_frame()
            for x in range(len(self.snake.body)):
                if self.snake.body[x].pos in list(map(lambda z: z.pos, self.snake.body[x + 1:])) or self.snake.head.pos[0] < 0 or self.snake.head.pos[0] > 19 or \
                        self.snake.head.pos[1] < 0 or self.snake.head.pos[1] > 19 or not self.steps:
                    self.reward=-1
                    self.score+=self.reward
                    #print('THIS IS THE END!!!')
                    #print(self.reward)
                    #print(self.score)
                    #print('Score: ', len(s.body)-1)
                    #message_box('You Lost', 'Play again')
                    self.next_state=None
                    print('end')
                    #return (self.reward,self.next_state)
                    #self.reset()
                    break
            
        self.steps-=1

        return (self.reward,self.next_state)

In [ ]:
q=deque(maxlen=5)
for i in range(5):
    q.append(i)
q
random.sample(q,2)

In [ ]:
class ReplayMemory(object):
    def __init__(self,capacity):
        self.memory=deque(maxlen=capacity)

    def push(self,*args):
        self.memory.append(Transition(*args))

    def sample(self,batch_size):
        return random.sample(self.memory,batch_size)

    def __len__(self):
        return len(self.memory)



In [ ]:
class DQN(nn.Module):
    def __init__(self,outputs):
        super(DQN,self).__init__()
        
        #self.conv1=nn.Conv2d(3,32,3,1,1)
        #self.bn1=nn.BatchNorm2d(32)
        
        #self.conv2=nn.Conv2d(32,64,3,1,1)
        #self.bn2=nn.BatchNorm2d(64)
        
        #self.conv3=nn.Conv2d(64,64,3,1,1)
        #self.bn3=nn.BatchNorm2d(64)

        #self.pool=nn.MaxPool2d(2)
        #self.pool1=nn.MaxPool2d(5)

        self.fc1=nn.Linear(20*20*3+4,512)
        self.fc2=nn.Linear(512,128)
        self.fc3=nn.Linear(128,64)
        self.fc4=nn.Linear(64,outputs)
        
    def forward(self,x):
        #print(x.shape)
        #x=F.relu(self.conv1(x))
        #print(x.shape)
        #x=F.relu(self.conv2(x))
        #print(x.shape)
        #x = self.pool(F.relu(self.conv3(x)))
        #print(x.shape)
        #print(x.shape)
        #x=x.view(-1,20*20*3)

        #x=torch.cat((x,features),1)
        #print(x.shape)
        #print(type(x))
        x=torch.sigmoid(self.fc1(x))
        x=torch.sigmoid(self.fc2(x))
        x=torch.sigmoid(self.fc3(x))
        x=self.fc4(x)
        
        return x

In [ ]:
def drawGrid(w,rows,surface):
    sizeBtwn=w//rows
    x=0
    y=0
    for l in range(rows):
        x+=sizeBtwn
        y+=sizeBtwn

        pygame.draw.line(surface,(255,255,255),(x,0),(x,w))
        pygame.draw.line(surface, (255, 255, 255), (0, y), (w, y))

def redrawWindow(surface,rows,width,s,snack):
    #global rows,width,s,snack
    surface.fill((0, 0, 0))
    s.draw(surface)
    snack.draw(surface)
    #drawGrid(width,rows,surface)
    pygame.display.update()

def randomSnack(rows,item):

    positions=item.body
    while True:
        x=random.randrange(rows)
        y=random.randrange(rows)
        if len(list(filter(lambda z:z.pos==(x,y),positions)))>0:
            continue
        else:
            break
    return (x,y)

def message_box(subject,content):
    root=tk.Tk()
    root.attributes("-topmost",True)
    root.withdraw()
    messagebox.showinfo(subject,content)
    try:
        root.destroy()
    except:
        pass

def get_screen(frame,feature):
    # the get frame of env returns H*W*C
    #get the screen of snake and change it to pytorch image format B*C*H*W
    #env.render()
    feature=torch.tensor(feature)/20.0
    if frame is None:
        return None
    screen=frame.transpose([2,0,1])
    screen = np.ascontiguousarray(screen, dtype=np.float32) / 255
    screen = torch.from_numpy(screen)
    # add .to(device) when train on GPU
    #return resize(screen).unsqueeze(0).to(device)
    screen=resize(screen)
    screen=screen.view(-1,20*20*3)
    #print(screen.shape)
    screen=torch.cat((screen,feature),1)
    #print(screen.shape)
    #screen=screen.unsqueeze(0)
    #print(screen.shape)
    return screen


In [ ]:
a=torch.arange(100000)
b=a.to(device)
del b
torch.cuda.empty_cache()


In [ ]:
device=torch.device('cuda')
device

In [ ]:
resize=T.Compose([T.ToPILImage(),
                  T.Resize(20, interpolation=Image.CUBIC),
                  T.ToTensor()])

Transition=namedtuple('Transition',
                        ('state', 'action', 'reward', 'next_state'))


BATCH_SIZE=16
GAMMA=0.99
EPS_START=0.9
EPS_END=0.1
EPS_DECAY=1000000
TARGET_UPDATE=10
NUM_ACTIONS=4
SCREEN_WIDTH=200
SCREEN_HEIGHT=200
steps_done=0

In [ ]:
del policy_net
del target_net
torch.cuda.empty_cache()

In [ ]:



#memory=ReplayMemory(50000)


#remember .to(device) for GPU
#feature=torch.tensor([[1,2]],dtype=torch.float32)
policy_net=DQN(NUM_ACTIONS).to(device)
target_net=DQN(NUM_ACTIONS).to(device)
target_net.load_state_dict(policy_net.state_dict())
#target_net.eval()




In [ ]:
optimizer=optim.Adam(policy_net.parameters(),lr=0.00001)

In [ ]:
target_net.eval()

In [ ]:
pygame.init()
env=Environment(SCREEN_WIDTH,SCREEN_HEIGHT,steps=50)
env.reset()
env.render()
s=env.get_curr_frame()

feature=torch.tensor([[1,2]],dtype=torch.float32)
s=get_screen(s,feature)
s_gpu=s.to(device)
s_gpu=torch.cat((s_gpu,s_gpu))
policy_net(s_gpu).max(1)[0]

In [ ]:
pygame.init()
env=Environment(SCREEN_WIDTH,SCREEN_HEIGHT,steps=50)
env.reset()
env.render()
s=env.get_curr_frame()
#print(resize(s).numpy().transpose((1,2,0)))
plt.figure(1)
plt.imshow(resize(s).numpy().transpose((1,2,0)))
print(env.snake.head.pos)
a=1
r,ns=env.step(a,'Agent')
plt.figure(2)
plt.imshow(ns)
print(env.snake.head.pos)
#s_torch=get_screen(s)
#s_torch=torch.cat([s_torch,s_torch,s_torch,s_torch],dim=1)
#out=policy_net(s_torch)
#s_torch.shape


In [ ]:
env.reset()
env.render()
s=env.get_curr_frame()

plt.figure(1)
plt.imshow(s)
print(env.snake.head.pos)
a=1
r,ns=env.step(a,'Agent')
plt.figure(2)
plt.imshow(ns)
print(env.snake.head.pos)

In [ ]:
a=input('enter the key: ')
print(typ)

In [ ]:
## behave like the Agent 0:up 1:down 2:left 3:right
pygame.init()
env=Environment(SCREEN_WIDTH,SCREEN_HEIGHT,steps=50)
env.reset()
env.render()
s=env.get_curr_frame()
plt.figure(1)
plt.clf()
plt.imshow(s)
#print('Snake pos: {}, Apple pos: {}'.format(env.snake.head.pos,env.snack.pos))


while env.snake.head.pos!=env.snack.pos:

    plt.figure(1)
    #plt.clf()
    plt.imshow(s)
    prev_snake_head_pos=env.snake.head.pos
    snack_pos=env.snack.pos
    
    plt.pause(0.001)
    a=input('enter the key: ')
    a=int(a)
    _,s=env.step(a,'Agent')
    print('Prev_pos: {}, Snake pos: {}, Apple pos: {}'.format(type(prev_snake_head_pos),env.snake.head.pos,snack_pos))
    

In [ ]:
##### a=np.array((1,2))-np.array((3,4))
np.sqrt(np.sum(a**2))

In [ ]:
non_final_mask

In [ ]:
stack.append(s_torch)
stack.append(s_torch)
stack.append(s_torch)
stack.append(s_torch)
s_torch=torch.cat(list(stack),dim=1)

In [ ]:
s_torch.shape
out=policy_net(s_torch)
out.max(1)[1].view(1, 1)

In [ ]:
def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.randint(4,(1,1))

In [ ]:
episode_durations = deque(maxlen=100)
scores=deque(maxlen=100)

def plot_all(fig_num=2):
    plt.figure(fig_num)
    plt.subplot(121)
    plt.clf()

    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(episode_durations)
    # Take 100 episode averages and plot them too
    if len(episode_durations) >= 100:
        means = np.mean(episode_durations)

        plt.plot(means)
    plt.pause(0.001)  # pause a bit so that plots are updated
    
    plt.subplot(122)
    plt.clf()

    plt.title('Score')
    plt.xlabel('Episode')
    plt.ylabel('Score')
    plt.plot(scores)
    if len(scores)>=100:
        mmeans=np.mean(scores)

        plt.plot(mmeans)
    plt.pause(0.001)
    
    

In [ ]:
plot_all()

In [ ]:
def optimize_model():
    gc.collect()
    if len(memory.memory)<BATCH_SIZE:
        return
#     if len(reward_memory.memory)<BATCH_SIZE:
#         return
#     if len(eat_memory.memory)<BATCH_SIZE:
#         return
    
    #a=int(BATCH_SIZE*0.5)
    #b=int(BATCH_SIZE*0.5)
#     c=int(BATCH_SIZE*0.2)
    
#     transitions=memory.sample(a)+reward_memory.sample(b)+eat_memory.sample(c)
    transitions=memory.sample(BATCH_SIZE)
    #print(type(transitions))
    random.shuffle(transitions)
    batch=Transition(*zip(*transitions))
    
    #device=device for GPU
    non_final_mask=torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)),device=device,dtype=torch.bool)
    #print('non final states mask: ',non_final_mask.shape)
    non_final_next_states=torch.cat([s for s in batch.next_state if s is not None]).to(device)
    #print('non final states: ',non_final_next_states.shape)
    state_batch=torch.cat(batch.state).to(device)
    #print('state_batch: ',state_batch.shape)
    action_batch=torch.cat(batch.action).to(device)
    #print('action_batch: ',action_batch.shape)
    reward_batch=torch.cat(batch.reward).to(device)
    #print('reward_batch: ',reward_batch)
    state_action_values=policy_net(state_batch).gather(1,action_batch)
    #print('state_action_values: ',state_action_values)
    next_state_values=torch.zeros(BATCH_SIZE).to(device)
    #print('next_state_values: ',next_state_values)
#     non_final_mask
#     non_final_next_states
#     state_batch
#     action_batch
    
    with torch.no_grad():
        next_state_values[non_final_mask]=target_net(non_final_next_states).max(1)[0]
        #print('next_state_values: ',next_state_values)
    expected_state_action_values=(next_state_values*GAMMA)+reward_batch
    #print('expected_state_action_values: ',expected_state_action_values.unsqueeze(1))
    loss=F.l1_loss(state_action_values,expected_state_action_values.unsqueeze(1))
    #print('losss: ',loss)
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1,1)
    optimizer.step()
    del non_final_mask
    del non_final_next_states
    del state_batch
    del action_batch
    del reward_batch
    del state_action_values
    del next_state_values
    torch.cuda.empty_cache()

In [ ]:
optimize_model()

In [ ]:
pygame.init()
env=Environment(SCREEN_WIDTH,SCREEN_HEIGHT,steps=50)
env.reset()
env.render()
s=env.get_curr_frame()

plt.imshow(s)

In [ ]:
transitions=memory.sample(int(BATCH_SIZE*0.75))+reward_memory.sample(int(BATCH_SIZE*0.25))
print(type(transitions))
random.shuffle(transitions)


In [ ]:
stack.append('a')
stack.append('b')
stack.append('c')
stack.append('d')
stack

In [ ]:
stack.append('e')
stack
torch.cat([s_torch,None])

In [ ]:
m=ReplayMemory(3)
m.push('a','b','c','d')
m.memory[-1]

In [ ]:
pygame.init()
env=Environment(SCREEN_WIDTH,SCREEN_HEIGHT,steps=200)

In [ ]:
init_episodes=500

num_episodes=20000

env.reward

In [ ]:
a=torch.tensor([1])
a.item()==1

In [ ]:
len(memory.sample(2)+memory.sample(3))

In [ ]:
episode_durations=[]
scores=[]
# frame_stack=4
memory=ReplayMemory(10000)
#reward_memory=ReplayMemory(10000)
#eat_memory=ReplayMemory(10000)

In [ ]:

# stack=deque(maxlen=frame_stack)
for i_init in range(init_episodes):
    env.reset()
    s=0
#     for i in range(frame_stack):
        
#         env.render()
#         state=env.get_curr_frame()
#         state=get_screen(state)
#         stack.append(state)
    #env.snack.pos
    for t in count():
        env.render()
        frame=env.get_curr_frame()
        state=get_screen(frame)
#         plt.figure(3)
#         plt.clf()
#         plt.imshow(frame)
#         plt.pause(0.001)
        #state=torch.cat(list(stack),dim=1)
        state_gpu=state.to(device)
        action=torch.randint(4,(1,1))
        
        prev_snake_pos=env.snake.head.pos
        snack_pos=env.snack.pos

        #print(action)
        reward,next_state=env.step(action.item(),'Agent')
        
            
        
        curr_snake_pos=env.snake.head.pos
        
        #small change should be positive if snake getting close to snack
        #else negative
        #prev_d=np.array(prev_snake_pos)-np.array(snack_pos)
        #prev_d=np.sqrt(np.sum(prev_d**2))
        prev_d=np.array(prev_snake_pos)-np.array(snack_pos)
        prev_d=np.sum(np.abs(prev_d))
        curr_d=np.array(curr_snake_pos)-np.array(snack_pos)
        curr_d=np.sum(np.abs(curr_d))
        
        small_change=(prev_d-curr_d)
        
        
        small_change=small_change*0.01 if small_change<0 else small_change*0.01
        
        small_change-=curr_d*0.001
        #print(small_change)
        #print(small_change)
        reward+=small_change
        #print(reward)
        #break
        #print(reward)
        s+=reward
        #plot_env()       
        reward=torch.tensor([reward],dtype=torch.float32)
        next_state=get_screen(next_state)
#         if next_state is not None:
#             stack.append(next_state)
#             next_state=torch.cat(list(stack),dim=1)
        #print(reward)
    
        if reward.item()>0:
            reward_memory.push(state,action,reward,next_state)
            
        memory.push(state,action,reward,next_state)

        optimize_model()

        del state_gpu
        torch.cuda.empty_cache()

        if next_state is None:
            episode_durations.append(t + 1)

            scores.append(s)

            plot_all(fig_num=2)

            break
    print(len(memory.memory))
    print(len(reward_memory.memory))
    #print(len(eat_memory.memory))
    
    if i_init%TARGET_UPDATE==0:
        target_net.load_state_dict(policy_net.state_dict())
print('initialization finished')
plt.show()

In [ ]:
len(reward_memory.memory)

In [ ]:
del memory

In [ ]:
(1,2)+(3,4)


In [ ]:
torch.tensor([(1,2)])

In [ ]:
#episode_durations=[]
#scores=[]
#frame_stack=4
#memory=ReplayMemory(10000)
#stack=deque(maxlen=frame_stack)

for i_episode in range(num_episodes):
    env.reset()
    env.render()
    s=0
#     for i in range(frame_stack):
#         env.render()
#         state=env.get_curr_frame()
#         state=get_screen(state)
#         stack.append(state)
    
    for t in count():
        
        frame=env.get_curr_frame()
        snack_pos=env.snack.pos
        
        prev_snake_pos=env.snake.head.pos
        
        state=get_screen(frame,[snack_pos+prev_snake_pos])
#         plt.figure(3)
#         plt.clf()
#         plt.imshow(frame)
#         plt.pause(0.001)
        #state=torch.cat(list(stack),dim=1)
        state_gpu=state.to(device)
        action=select_action(state_gpu)
        action=action.cpu()
        #print(action)

        
        
        
        reward,next_state=env.step(action.item(),'Agent')
        
        
        curr_snake_pos=env.snake.head.pos
        
        #small change should be positive if snake getting close to snack
        #else negative
        
        prev_d=np.array(prev_snake_pos)-np.array(snack_pos)
        prev_d=np.sqrt(np.sum(prev_d**2))
        curr_d=np.array(curr_snake_pos)-np.array(snack_pos)
        curr_d=np.sqrt(np.sum(curr_d**2))
        #small_change=(prev_d-curr_d)
        #small_change=small_change*0.01 
        if prev_d>=curr_d:
            reward+=0.01*np.abs(0.99*curr_d-prev_d)
        else:
            reward-=0.02*np.abs(0.99*curr_d-prev_d)

        
        #small_change-=curr_d*0.001
        #print(small_change)
        
        
        s+=reward
        #plot_env()       
        reward=torch.tensor([reward],dtype=torch.float32)
        

        
        next_state=get_screen(next_state,[snack_pos+ curr_snake_pos])
#         if reward.item()>0:
#             reward_memory.push(state,action,reward,next_state)
#         if next_state is not None:
#             stack.append(next_state)
#             next_state=torch.cat(list(stack),dim=1)
        #print(next_state.shape)
        memory.push(state,action,reward,next_state)
        
        optimize_model()
        
        del state_gpu
        torch.cuda.empty_cache()

        if next_state is None:
            episode_durations.append(t + 1)

            scores.append(s)
            
            plot_all(fig_num=3)

            break
    if i_episode%TARGET_UPDATE==0:
        target_net.load_state_dict(policy_net.state_dict())
    if i_episode%100==0:
        torch.save(policy_net.state_dict(),'snake')
            
print('Complete')
plt.show()
        

In [ ]:
torch.save(policy_net.state_dict(),'snake')
import pickle 
with open('eps_duration.txt','wb') as fp:
    pickle.dump(episode_durations,fp)
with open('score.txt','wb') as fp:
    pickle.dump(scores,fp)

In [ ]:
for trans in reward_memory.memory:
    if trans.next_state is np.ndarray:
        reawrd_memory.remove(trans)
for trans in memory.memory:
    if trans.next_state is np.ndarray:
        memory.remove(trans)

In [ ]:
for i_episode in range(num_episodes):
    env.reset()
    s=0
#     for i in range(frame_stack):
#         env.render()
#         state=env.get_curr_frame()
#         state=get_screen(state)
#         stack.append(state)
    for t in count():
        frame=env.get_curr_frame()
        snack_pos=env.snack.pos
        
        prev_snake_pos=env.snake.head.pos
        
        state=get_screen(frame,[snack_pos+prev_snake_pos])
        plt.figure(3)
        plt.clf()
        plt.imshow(frame)
        plt.pause(0.001)

        state_gpu=state.to(device)
        action=select_action(state_gpu)
        action=action.cpu()
        #print(action)

        
        
        
        reward,next_state=env.step(action.item(),'Agent')
        
        
        curr_snake_pos=env.snake.head.pos

        #small change should be positive if snake getting close to snack
        #else negative

        prev_d=np.array(prev_snake_pos)-np.array(snack_pos)
        prev_d=np.sqrt(np.sum(prev_d**2))
        curr_d=np.array(curr_snake_pos)-np.array(snack_pos)
        curr_d=np.sqrt(np.sum(curr_d**2))
        #small_change=(prev_d-curr_d)
        #small_change=small_change*0.01 
        if prev_d>=curr_d:
            reward+=0.01*np.abs(0.99*curr_d-prev_d)
        else:
            reward-=0.02*np.abs(0.99*curr_d-prev_d)

        
        #small_change-=curr_d*0.001
        #print(small_change)
        
        
        s+=reward
        #plot_env()       
        reward=torch.tensor([reward],dtype=torch.float32)
        

        
        next_state=get_screen(next_state,[snack_pos+ curr_snake_pos])
#         if reward.item()>0:
#             reward_memory.push(state,action,reward,next_state)
#         if next_state is not None:
#             stack.append(next_state)
#             next_state=torch.cat(list(stack),dim=1)
        #print(next_state.shape)
        if next_state is not None:
            pass
#             stack.append(next_state)
#             next_state=torch.cat(list(stack),dim=1)
        else:
            print(env.steps)
            break

In [ ]:
torch.cuda.empty_cache()

In [ ]:
x=[s for s in batch.next_state if s is not None]
for t in x:
    print(t.shape)

In [ ]:
non_final_mask